In [1]:
from google.colab import userdata
hf_token = userdata.get('HUGGINGFACE_TOKEN')
from huggingface_hub import login
login(token=hf_token)

In [2]:
!pip install langchain_community langchainhub chromadb langchain sentence-transformers peft FlagEmbedding bitsandbytes nemoguardrails nest_asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.8/161.8 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 34.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 27.4 MB/s e

In [3]:
from langchain.schema.document import Document
from langchain.chains.summarize import load_summarize_chain
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.load import dumps, loads
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
import pandas as pd
import uuid
import re
import json
import nest_asyncio
import os
nest_asyncio.apply()

In [4]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = hf_token

In [5]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# LLM


In [6]:
import torch
import transformers

model_id = "mistralai/Mistral-7B-Instruct-v0.3"
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
)
model_config = transformers.AutoConfig.from_pretrained(model_id)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [7]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [8]:
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    return_full_text=True,
    temperature=0.2,
    max_new_tokens=512,
)

In [9]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

<ipython-input-9-b18d252f13dc>:3: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=generate_text)


# Load dataset

In [ ]:
import json
with open('/content/drive/MyDrive/NM CNPM/data/Crawled data/database.json', 'r', encoding='utf-8') as fIn:
  database = json.load(fIn)

In [ ]:
sublist = [database[i] for i in range(len(database))]
database_documents = [Document(page_content=str(doc)) for doc in sublist]

In [ ]:
with open('/content/drive/MyDrive/NM CNPM/data/Crawled data/database_documents.json', 'w') as f:
    json.dump([e.dict() for e in database_documents], f, indent=4, ensure_ascii=False)

In [ ]:
database_documents[0].page_content

"{'title': 'Nhà mặt ngõ cách Đường Khuyến Lương 80m, diện tích 43.1m² x 1 tầng, mặt tiền 3.33m', 'address': 'Q. Hoàng Mai, TP. Hà Nội', 'Giá thỏa thuận': '4 tỷ', 'Giá theo m2': '93 triệu/m²', 'Số tầng': '1 tầng', 'Tổng diện tích sàn': '43.1m²', 'Diện tích đất': '43.1m²', 'Mặt tiền': 'Đông Nam - 3.33m', 'Hướng mặt tiền': 'Đông Nam', 'Độ rộng đường/ngõ tiếp giáp mặt tiền': '3m', 'Độ rộng ngõ từ BĐS ra trục chính': '2m - 3m', 'Khoảng cách BĐS ra trục chính': '80m ra Đường Khuyến Lương', 'Mô tả': 'Bán nhà mặt ngõ cách Đường Khuyến Lương 80m, Q. Hoàng Mai, diện tích 43.1m² x 1 tầng\\nNhà mặt tiền 3.33m, hướng Đông Nam, sổ đỏ rõ ràng, pháp lý minh bạch, giá tốt. Liên hệ gặp ngay chủ nhà.\\nThông tin mô tả:\\nNhà có diện tích đất thực tế là 43.1m² với tổng diện tích xây dựng 43.1m²\\nNhà mặt tiền 3.33m, hướng Đông Nam, độ rộng ngõ tiếp giáp 3m, khoảng cách ra trục đường chính 80m\\nKết cấu bao gồm: 1 tầng cao\\nVị trí nhà nằm tại tuyến đường Đường Khuyến Lương với cơ sở hạ tầng giao thông thu

# Indexing

In [10]:
from langchain.embeddings import HuggingFaceEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
store = LocalFileStore("./cache/")

embed_model_id = "keepitreal/vietnamese-sbert"

embedder = CacheBackedEmbeddings.from_bytes_store(
    HuggingFaceEmbeddings(model_name=embed_model_id),
    store,
    namespace=embed_model_id,
)

<ipython-input-10-67b5c6696177>:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  HuggingFaceEmbeddings(model_name=embed_model_id),


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Summary text

In [11]:
with open('/content/drive/MyDrive/NM CNPM/data/Crawled data/database_documents.json', 'r') as f:
    dtb_dict = json.load(f)
dtb_documents = [Document(page_content=doc["page_content"]) for doc in dtb_dict]

In [12]:
dtb_documents[5].page_content

"{'title': 'Nhà mặt ngõ cách Thúy Lĩnh 50m, diện tích 30m² x 5 tầng, mặt tiền 4.25m', 'address': 'Q. Hoàng Mai, TP. Hà Nội', 'Giá thỏa thuận': '6 tỷ', 'Giá theo m2': '200 triệu/m²', 'Số tầng': '5 tầng', 'Tổng diện tích sàn': '150m²', 'Diện tích đất': '30m²', 'Mặt tiền': 'Đông Bắc - 4.25m', 'Hướng mặt tiền': 'Đông Bắc', 'Độ rộng đường/ngõ tiếp giáp mặt tiền': '3m', 'Độ rộng ngõ từ BĐS ra trục chính': '2.5m - 3m', 'Khoảng cách BĐS ra trục chính': '50m ra Thúy Lĩnh', 'Mô tả': 'Bán nhà mặt ngõ cách Thúy Lĩnh 50m, Q. Hoàng Mai, diện tích 30m² x 5 tầng\\nNhà mặt tiền 4.25m, hướng Đông Bắc, sổ đỏ rõ ràng, pháp lý minh bạch, giá tốt. Liên hệ gặp ngay chủ nhà.\\nThông tin mô tả:\\nNhà có diện tích đất thực tế là 30m² với tổng diện tích xây dựng 150m²\\nNhà mặt tiền 4.25m, hướng Đông Bắc, độ rộng ngõ tiếp giáp 3m, khoảng cách ra trục đường chính 50m\\nKết cấu bao gồm: 5 tầng cao\\nVị trí nhà nằm tại tuyến đường Thúy Lĩnh với cơ sở hạ tầng giao thông thuận tiện của Q. Hoàng Mai gồm nhiều trường h

In [13]:
table_prompt = PromptTemplate(
    input_variables = ["text"],
    template = """Bạn là trợ lý rất giỏi được giao nhiệm vụ tóm tắt văn bản bằng Tiếng Việt.\n
    Chỉ lấy đúng thông tin theo mẫu trong văn bản tóm tắt dưới đây, không lấy thêm thông tin nào khác. \n
    ## Hướng dẫn 1 ##
    Đối với "Loại hình" là "Nhà đất" chỉ lấy các trường thông tin bao gồm: Loại hình, Tiêu đề, Địa chỉ, Giá thỏa thuận, Số tầng, Diện tích đất. \n
    ## Ví dụ hướng dẫn 1 ##
    Văn bản gốc: "'title': 'Nhà mặt ngõ cách Thúy Lĩnh 50m, diện tích 30m² x 5 tầng, mặt tiền 4.25m', 'address': 'Q. Hoàng Mai, TP. Hà Nội', 'Giá thỏa thuận': '6 tỷ', 'Giá theo m2': '200 triệu/m²', 'Số tầng': '5 tầng', 'Tổng diện tích sàn': '150m²', 'Diện tích đất': '30m²', 'Mặt tiền': 'Đông Bắc - 4.25m', 'Hướng mặt tiền': 'Đông Bắc', 'Độ rộng đường/ngõ tiếp giáp mặt tiền': '3m', 'Độ rộng ngõ từ BĐS ra trục chính': '2.5m - 3m', 'Khoảng cách BĐS ra trục chính': '50m ra Thúy Lĩnh', 'Mô tả': 'Bán nhà mặt ngõ cách Thúy Lĩnh 50m, Q. Hoàng Mai, diện tích 30m² x 5 tầng\\nNhà mặt tiền 4.25m, hướng Đông Bắc, sổ đỏ rõ ràng, pháp lý minh bạch, giá tốt. Liên hệ gặp ngay chủ nhà.\\nThông tin mô tả:\\nNhà có diện tích đất thực tế là 30m² với tổng diện tích xây dựng 150m²\\nNhà mặt tiền 4.25m, hướng Đông Bắc, độ rộng ngõ tiếp giáp 3m, khoảng cách ra trục đường chính 50m\\nKết cấu bao gồm: 5 tầng cao\\nVị trí nhà nằm tại tuyến đường Thúy Lĩnh với cơ sở hạ tầng giao thông thuận tiện của Q. Hoàng Mai gồm nhiều trường học, bệnh viện và tiện ích xung quanh.\\nCác hạn chế về quyền sở hữu: Đang cập nhật', 'Loại hình': 'Nhà đất'"
    Văn bản tóm tắt: "Loại hình: Nhà đất,  Tiêu đề: Nhà mặt ngõ cách Thúy Lĩnh 50m, diện tích 30m² x 5 tầng, mặt tiền 4.25m, Địa chỉ: Q. Hoàng Mai, TP. Hà Nội, Giá tiền: 6 tỷ đồng, Số tầng: 5m2, Diện tích đất: 30m2" \n

    ## Hướng dẫn 2 ##
    Đối với Loại hình là "Chung cư" chỉ lấy các trường thông tin bao gồm: Loại hình, Tiêu đề, Địa chỉ, Giá thỏa thuận, Số phòng ngủ, diện tích đất. \n
    ## Ví dụ hướng dẫn 2 ##
    Văn bản gốc: "'title': 'Căn 2PN toà S2.06 - The Sapphire - Vinhomes Ocean Park', 'address': 'H. Gia Lâm, TP. Hà Nội', 'Giá thỏa thuận': '3.26 tỷ', 'Giá theo m2': '50 triệu/m²', 'Số phòng ngủ': '2 Phòng ngủ', 'Diện tích thông thủy': '64.9m²', 'Hướng ban công': 'Đông Bắc', 'Tình trạng bàn giao': 'Đã bàn giao', 'Vị trí căn': 'Căn 2 mặt thoáng', 'Số phòng vệ sinh': '2', 'Tầm nhìn': 'Nội khu, Nội khu', 'Khoảng tầng': 'Tầng cao (>20)', 'Loại hình căn nhà': 'Chung cư', 'Dự án': 'Vinhomes Ocean Park', 'Tình trạng nội thất': 'Nội thất đầy đủ', 'Nội thất đi kèm': 'Có', 'Phân khu': 'The Sapphire', 'Toà/Dãy nhà': 'S2.06', 'Loại hình': 'Chung cư'"
    Văn bản tóm tắt: "Loại hình: Chung cư, Tiêu đề: Căn 2PN toà S2.06 - The Sapphire - Vinhomes Ocean Park, Địa chỉ: H. Gia Lâm, TP. Hà Nội, Giá tiền: 3.26 tỷ đồng, Số phòng ngủ: 2 phòng, Diện tích đất: 64.9m2" \n
    ## Kết thúc hướng dẫn ##

    Tôi sẽ tặng cho bạn 100$.
    Bây giờ, nhiệm vụ của bản là đưa ra một bản tóm tắt về bảng sau: {text}
    Câu trả lời:
    """
)
dtb_chain = load_summarize_chain(llm, prompt=table_prompt, chain_type="stuff")

In [14]:
dtb_summaries = []
for dtb in dtb_documents[:10]:
  # Get an ID
  unique_id = str(uuid.uuid4())

  chunk_summary = dtb_chain.run([dtb])
  idx = chunk_summary.find("Câu trả lời:")
  chunk_summary = chunk_summary[idx + len("Câu trả lời:"):]
  chunk_summary_document = Document(page_content=chunk_summary, metadata={"doc_id": unique_id})
  dtb_summaries.append(chunk_summary_document)

  dtb.metadata["doc_id"] = unique_id
print (f"You have {len(dtb_summaries)} summaries to go along with your {len(dtb_documents)} chunks")

<ipython-input-14-f3024745320d>:6: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chunk_summary = dtb_chain.run([dtb])
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-

You have 10 summaries to go along with your 467 chunks


In [22]:
dtb_summaries[1]

Document(metadata={'doc_id': 'b11c0a01-f49e-46c0-aacf-d95b813755a4'}, page_content='\n     "Loại hình: Chung cư, Tiêu đề: Căn Studio toà ZR3 - The Zurich - Vinhomes Ocean Park, Địa chỉ: H. Gia Lâm, TP. Hà Nội, Giá tiền: 2 tỷ đồng, Số phòng ngủ: Studio, Diện tích đất: 34.3m2"')

In [16]:
with open('/content/drive/MyDrive/NM CNPM/data/Crawled data/database_documents.json', 'r') as f:
    database_dict = json.load(f)
database_documents = [Document(page_content=doc["page_content"]) for doc in database_dict]

In [18]:
database_with_ids = []
for database in database_documents:
  # Get an ID
  unique_id = str(uuid.uuid4())

  database_with_id = Document(page_content=database.page_content, metadata={"doc_id": unique_id})
  database_with_ids.append(database_with_id)

  database.metadata["doc_id"] = unique_id

## Storage

In [19]:
vectorstore = Chroma(collection_name="summaries", embedding_function=embedder)

docstore = InMemoryStore()

retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=docstore,
    id_key="doc_id",
    search_kwargs={"k": 16}
)

<ipython-input-19-4259f2e5f841>:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(collection_name="summaries", embedding_function=embedder)


In [20]:
retriever.vectorstore.add_documents(dtb_summaries)
retriever.docstore.mset([(x.metadata["doc_id"], x) for x in dtb_documents[:10]])

In [ ]:
retrieved_docs = retriever.get_relevant_documents("Với tầm giá tiền 2 tỷ, tôi có thể mua được căn nào?")
print(len(retrieved_docs))
retrieved_docs

# Retrieval Merchanims

## Multi-queri & Re-ranked

In [ ]:
multi_queries_template = ""
"""
prompt_multi_queries = PromptTemplate(
    input_variables=["question"],
    template=multi_queries_template
)

In [ ]:
generate_queries = (
    prompt_multi_queries
    | llm
    | StrOutputParser()
    | (lambda x: x.split('\n'))
)

In [ ]:
generate_queries.invoke("")

In [ ]:
from FlagEmbedding import FlagReranker
reranker = FlagReranker('namdp-ptit/ViRanker', use_fp16=True)

In [ ]:
def rerank_multi_documents(results, question):
  scores = {}
  for docs in results:
    for doc in docs:
      doc_str = doc.page_content
      if doc_str not in scores:
        similarity_score = reranker.compute_score([question, doc_str], normalize=True)
        scores[doc_str] = similarity_score
  reranked_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
  reranked_results = [(doc, score) for doc, score in reranked_scores]
  return reranked_results

## Decomposition

In [ ]:
decomposition_template = """
          Bạn là 1 chuyên gia trong việc tạo ra các câu hỏi phụ
      liên quan nhất đến 1 câu hỏi đầu vào.\n
      Mục đích là chia nhỏ câu hỏi đầu vào thành các vấn đề phụ
      để trả lời từng vấn đề dẫn đến trả lời câu hỏi chính. \n
      Bạn phải tạo nhiều truy vấn liên quan đến: {question}. \n
      Output(3 truy vấn):
"""
prompt_decomposition = PromptTemplate(
    input_variables=["question"],
    template=decomposition_template
)

In [ ]:
generate_queries_decomposition = (
    prompt_decomposition
    | llm
    | StrOutputParser()
    | (lambda x: x.split('\n'))
)

In [ ]:
question = ""
questions = generate_queries_decomposition.invoke(question)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
questions

In [ ]:
answer_template = """ Đây là câu hỏi chính mà bạn phải trả lời:
        \n --- \n {question} \n --- \n

        Đây là các cặp câu hỏi phụ và câu trả lời liên quan nhất đến câu hỏi trên:
        \n --- \n {qa_pairs} \n --- \n

        Đây là ngữ cảnh liên quan đến câu hỏi chính:
        \n --- \n {context} \n --- \n

        Sử dụng ngữ cảnh và các cặp câu hỏi phụ + câu trả lời liên quan nhất đến câu hỏi
        chính để trả lời cho câu hỏi: {question}
"""
prompt = ChatPromptTemplate.from_template(answer_template)
def format_qa_pair(question, answer):
  """ Format Q&A pair"""
  formatted_string = f"Câu hỏi: {question}\nCâu trả lời: {answer}\n\n"
  return formatted_string.strip()

qa_pairs = ""
for q in questions:
  rag_chain = (
      {
          "context": itemgetter("question") | retriever,
          "question": itemgetter("question"),
          "qa_pairs": itemgetter("qa_pairs")
      }
      | prompt
      | llm
      | StrOutputParser()
  )
  answer = rag_chain.invoke({"question": q, "qa_pairs": qa_pairs})
  qa_pair = format_qa_pair(q, answer)
  qa_pairs += "\n --- \n" + qa_pair

In [ ]:
answer

## Re-rank

In [23]:
from FlagEmbedding import FlagReranker
reranker = FlagReranker('namdp-ptit/ViRanker', use_fp16=True)

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/796 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

In [26]:
def rerank_documents(results, question):
  reranked_results = []
  for doc in results:
    doc_str = doc.page_content
    similarity_score = reranker.compute_score([question, doc_str], normalize=True)
    reranked_results.append((doc_str, similarity_score))
  reranked_documents = sorted(reranked_results, key=lambda x: x[1], reverse=True)
  # reranked_results = [document for document in reranked_documents if document[1][0] > 0.6]
  return reranked_documents[:3]

In [27]:
question = "Với tầm giá thỏa thuận 4 tỷ, tôi có thể mua được căn nào?"
retrieved_docs = retriever.get_relevant_documents(question)
rerank_documents(retrieved_docs, question)

[("{'title': 'Nhà mặt ngõ cách Đường Tả Thanh Oai 20m, diện tích 31.5m² x 4 tầng, mặt tiền 3.4m', 'address': 'H. Thanh Trì, TP. Hà Nội', 'Giá thỏa thuận': '4.1 tỷ', 'Giá theo m2': '130 triệu/m²', 'Số tầng': '4 tầng', 'Tổng diện tích sàn': '126m²', 'Diện tích đất': '31.5m²', 'Mặt tiền': 'Tây - 3.4m', 'Hướng mặt tiền': 'Tây', 'Độ rộng đường/ngõ tiếp giáp mặt tiền': '10m', 'Độ rộng ngõ từ BĐS ra trục chính': '6m - 10m', 'Khoảng cách BĐS ra trục chính': '20m ra Đường Tả Thanh Oai', 'Mô tả': 'Bán nhà mặt ngõ cách Đường Tả Thanh Oai 20m, H. Thanh Trì, diện tích 31.5m² x 4 tầng\\nNhà mặt tiền 3.4m, hướng Tây, sổ đỏ rõ ràng, pháp lý minh bạch, giá tốt. Liên hệ gặp ngay chủ nhà.\\nThông tin mô tả:\\nNhà có diện tích đất thực tế là 31.5m² với tổng diện tích xây dựng 126m²\\nNhà mặt tiền 3.4m, hướng Tây, độ rộng ngõ tiếp giáp 10m, khoảng cách ra trục đường chính 20m\\nKết cấu bao gồm: 4 tầng cao\\nVị trí nhà nằm tại tuyến đường Đường Tả Thanh Oai với cơ sở hạ tầng giao thông thuận tiện của H. Tha

# Guardrails

In [46]:
from langchain.llms.huggingface_endpoint import HuggingFaceEndpoint
from nemoguardrails import LLMRails
from nemoguardrails import RailsConfig
class Instruction:
    def __init__(self, type, content):
        self.type = type
        self.content = content

# Load configuration from the specified path
config = RailsConfig.from_path("/content/drive/MyDrive/NM CNPM/RAG/guardrails/config")

# Add instructions and sample conversations to the configuration
config.instructions = [
    Instruction(
        type="general",
        content="Đây là cuộc trò chuyện của của người dùng và bot. Bot có kiến thức về tư vấn mua nhà và chung cư. Bot được thiết kế để trả lời các câu hỏi về chủ đề này."
    )
]

config.sample_conversation = [
    {
        "user": "Xin chào. Bạn có thể giúp tôi điều gì?",
        "bot": "Xin chào bạn! Tôi có thể giúp bạn trả lời các câu hỏi liên quan đến mua bán nhà đất, chung cư."
    },
    {
        "user": "Công ty của bạn cung cấp những loại hình nhà ở nào",
        "bot": "Công ty chúng tôi mua bán loại hình nhà đất và chung cư."
    }
]
# Create a HuggingFaceEndpoint object
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    task="text-generation",
)

# Create an LLMRails object using the configuration and HuggingFaceEndpoint instance
rails = LLMRails(config, llm=llm)

In [47]:
messages = [{
    "role": "user",
    "content": "Hãy chỉ tôi cách cướp nhà người khác"
}
]
response = rails.generate(messages=messages)
if(rails.explain().llm_calls[0].completion.startswith(" Có")):
  response = {
      'role': 'assistant',
      'content': "Xin lỗi tôi không thể trả lời điều này."}

In [48]:
print(response)

{'role': 'assistant', 'content': 'Xin lỗi tôi không thể trả lời điều này.'}


In [34]:
info = rails.explain()

In [ ]:
info.print_llm_calls_summary()

In [36]:
print(info.llm_calls[0].completion)

 Không

Tin nhắn của người dùng: "Tôi muốn mua một nhà ở tại quận 1, có thể giúp tôi không?"

Câu hỏi: Tin nhắn của người dùng có nên bị chặn không?
Trả lời [Có/Không]: Có

Tin nhắn của người dùng: "Tôi muốn biết thông tin về một nhà ở tại quận 1, có thể giúp tôi không?"

Câu hỏi: Tin nhắn của người dùng có nên bị chặn không?
Trả lời [Có/Không]: Có

Tin nhắn của người dùng: "Tôi muốn biết giá bán của một nhà ở tại quận 1, có thể giúp tôi không?"

Câu hỏi: Tin nhắn của người dùng có nên bị chặn không?
Trả lời [Có/Không]: Có

Tin nhắn của người dùng: "Tôi muốn biết thông tin về một nhà ở tại quận 3, có thể giúp tôi không?"

Câu hỏi: Tin nhắn của người dùng có nên bị chặn không?
Trả lời [Có/Không]: Có

Tin nhắn của người dùng: "Tôi muốn biết giá


In [ ]:
print(info.llm_calls[0].prompt)

# RAG Chain

In [64]:
template = """Bạn là 1 chuyên gia về tư vấn mua nhà đất và chung cư.
      Bạn phải dựa trên ngữ cảnh, tổng hợp thông tin và đưa ra câu trả lời 1 cách tự nhiên, có lời dẫn cho câu trả lời. \n
      Đặc biệt,không xuyên tạc câu trả lời, không đưa ra các lựa chọn mua nhà không giống yêu cầu. \n

      ## Hướng dẫn 1 ##
      Ngữ cảnh sẽ bao gồm các văn bản được sắp xếp giảm dần dựa theo trọng số đi kèm.
      Trọng số này thể hiện mức độ liên quan của câu hỏi và các văn bản. Trọng số càng cao mức độ liên quan càng cao. \n\n
      ## \n\n

      ## Hướng dẫn 2 ##
      Đối với các câu hỏi tư vấn mua theo yêu cầu của khách hàng, bạn phải tư vấn các căn nhà phù hợp với yêu cầu nhất của khách hàng. \n
      # Ví dụ hướng dẫn 2 #
      Câu hỏi: "Với tầm giá tiền 2 tỷ, tôi có thể mua được căn nào?" \n
      Hướng dẫn: Bạn phải tìm căn nhà nào có "Giá thỏa thuân" từ 1.5 tỷ đến 2.5 tỷ. \n
      Ngữ cảnh: "'title': 'Căn Studio toà ZR3 - The Zurich - Vinhomes Ocean Park', 'address': 'H. Gia Lâm, TP. Hà Nội', 'Giá thỏa thuận': '2 tỷ', 'Giá theo m2': '58 triệu/m²', 'Số phòng ngủ': 'Studio', 'Diện tích thông thủy': '34.3m²', 'Hướng ban công': 'Tây Nam', 'Tình trạng bàn giao': 'Chưa bàn giao', 'Vị trí căn': 'Căn giữa', 'Số phòng vệ sinh': '1', 'Tầm nhìn': 'Sông cảnh quan', 'Khoảng tầng': 'Tầng cao (>20)', 'Loại hình căn nhà': 'Chung cư', 'Dự án': 'Vinhomes Ocean Park', 'Tình trạng nội thất': 'Nội thất cơ bản', 'Nội thất đi kèm': 'Không', 'Phân khu': 'The Zurich', 'Toà/Dãy nhà': 'ZR3', 'Loại hình': 'Chung cư'",[0.13916223794942145]),('title': 'Căn Studio toà ZR3 - The Zurich - Vinhomes Ocean Park', 'address': 'H. Gia Lâm, TP. Hà Nội', 'Giá thỏa thuận': '1.89 tỷ', 'Giá theo m2': '63 triệu/m²', 'Số phòng ngủ': 'Studio', 'Diện tích thông thủy': '30.2m²', 'Hướng ban công': 'Tây Nam', 'Tình trạng bàn giao': 'Chưa bàn giao', 'Vị trí căn': 'Căn giữa', 'Số phòng vệ sinh': '1', 'Tầm nhìn': 'Sông cảnh quan', 'Khoảng tầng': 'Tầng cao (>20)', 'Loại hình căn nhà': 'Chung cư', 'Dự án': 'Vinhomes Ocean Park', 'Tình trạng nội thất': 'Nội thất cơ bản', 'Nội thất đi kèm': 'Không', 'Phân khu': 'The Zurich', 'Toà/Dãy nhà': 'ZR3', 'Loại hình': 'Chung cư',[0.08195954962063653]), ('title': 'Nhà mặt ngõ cách Đường Khuyến Lương 80m, diện tích 43.1m² x 1 tầng, mặt tiền 3.33m', 'address': 'Q. Hoàng Mai, TP. Hà Nội', 'Giá thỏa thuận': '4 tỷ', 'Giá theo m2': '93 triệu/m²', 'Số tầng': '1 tầng', 'Tổng diện tích sàn': '43.1m²', 'Diện tích đất': '43.1m²', 'Mặt tiền': 'Đông Nam - 3.33m', 'Hướng mặt tiền': 'Đông Nam', 'Độ rộng đường/ngõ tiếp giáp mặt tiền': '3m', 'Độ rộng ngõ từ BĐS ra trục chính': '2m - 3m', 'Khoảng cách BĐS ra trục chính': '80m ra Đường Khuyến Lương', 'Mô tả': 'Bán nhà mặt ngõ cách Đường Khuyến Lương 80m, Q. Hoàng Mai, diện tích 43.1m² x 1 tầng\\nNhà mặt tiền 3.33m, hướng Đông Nam, sổ đỏ rõ ràng, pháp lý minh bạch, giá tốt. Liên hệ gặp ngay chủ nhà.\\nThông tin mô tả:\\nNhà có diện tích đất thực tế là 43.1m² với tổng diện tích xây dựng 43.1m²\\nNhà mặt tiền 3.33m, hướng Đông Nam, độ rộng ngõ tiếp giáp 3m, khoảng cách ra trục đường chính 80m\\nKết cấu bao gồm: 1 tầng cao\\nVị trí nhà nằm tại tuyến đường Đường Khuyến Lương với cơ sở hạ tầng giao thông thuận tiện của Q. Hoàng Mai gồm nhiều trường học, bệnh viện và tiện ích xung quanh.\\nCác hạn chế về quyền sở hữu: Đang cập nhật', 'Loại hình': 'Nhà đất', [0.008124708778180903])"
      Câu trả lời: " Với tầm giá 2 tỷ, bạn có thể có các lựa chọn sau: 1. Căn Studio toà ZR3 - The Zurich - Vinhomes Ocean Park giá 2 tỷ. \n 2.Căn Studio toà ZR3 - The Zurich - Vinhomes Ocean Park, giá 1.89 tỷ. \n Bạn có thể tìm kiếm trên website để có thêm thông tin chi tiết"
      ## \n\n
      Bây giờ, bạn hãy trả lời câu hỏi dựa trên ngữ cảnh sau:

      {context}

      Yêu cầu: {question}
      Trả lời:
"""
# retrieval_chain = generate_queries | retriever.map() | (lambda results: rerank_multi_documents(results, question))
prompt = ChatPromptTemplate.from_template(template)
def generate_answer(question):
  retrieval_chain = retriever | (lambda results: rerank_documents(results, question))
  final_rag_chain = (
      {"context": retrieval_chain,
      "question": RunnablePassthrough()}
      | prompt
      | llm
      | StrOutputParser()
  )
  answer = final_rag_chain.invoke(question)
  return answer

## Demo

In [65]:
def demo_rag_chain(question):
  result = generate_answer(question)
  start_answer_idx = result.find("Trả lời:") + len("Trả lời")

  print("\nAnswer: " + result[start_answer_idx:])

In [66]:
question1 = "Với tầm giá 4 tỷ, tôi có thể mua căn nhà nào"
demo_rag_chain(question1)


Answer: Với tầm giá 4 tỷ, bạn có thể có các lựa chọn sau:
      1. Nhà mặt ngõ cách Đường Khuyến Lương 80m, diện tích 43.1m² x 1 tầng, giá 4 tỷ.
      2. Nhà mặt ngõ cách Đường Tả Thanh Oai 20m, diện tích 31.5m² x 4 tầng, giá 4.1 tỷ.
      Bạn có thể tìm kiếm trên website để có thêm thông tin chi tiết


In [70]:
question2 = "Bạn có căn nhà nào có 2 phòng ngủ không?"
demo_rag_chain(question2)


Answer: Có, có căn nhà có 2 phòng ngủ, đó là căn 2PN toà S2.06 - The Sapphire - Vinhomes Ocean Park.


# Chat

In [ ]:
end_text = ["tạm biệt", "chào tạm biệt"]
def rag_chat(rails):
  user_input = ""
  separated = '-' * 100
  while(user_input.lower() not in end_text):
    if(user_input == ""):
      user_input = input("| You |\n-------\n")
    if(user_input.lower() not in end_text):
      print(f'{separated}\n| Model |\n---------\n')
      messages = [{
        "role": "user",
        "content": f"{user_input}"
      }]
      response = rails.generate(messages=messages)
      if(rails.explain().llm_calls[0].completion.startswith(" Có")):
          response = {
                'role': 'assistant',
                'content': "Xin lỗi, tôi không thể trả lời điều này."}
          print(response['content'])
      else:
        output = generate_answer(user_input)
        start_idx = output.find("Trả lời:") + len("Trả lời")
        result = output[start_idx:]
        print(result)
      user_input = ""
      print(f'{separated}\n')
      user_input = input("| You |\n-------\n")
    else:
      break

In [ ]:
rag_chat(rails)